In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

###Step 1: Loading Video

In [3]:
video_path = '/content/spatio.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not load the video.")
else:
    print("Video loaded successfully.")


Video loaded successfully.


###Step 2: Frame Extraction

In [4]:
frame_dir = '/content/ext_frames'
os.makedirs(frame_dir, exist_ok=True)

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("No frame captured or end of video reached.")
        break

    frame_filename = os.path.join(frame_dir, f"frame_{frame_count}.jpg")
    cv2.imwrite(frame_filename, frame)

    frame_count += 1

print(f"Total frames extracted: {frame_count}")
cap.release()

No frame captured or end of video reached.
Total frames extracted: 912


###Step 3: Spatio-Temporal Segmentation

In [5]:
segmented_frames_dir = '/content/segment_frames'
os.makedirs(segmented_frames_dir, exist_ok=True)

for i in range(frame_count):
    frame = cv2.imread(os.path.join(frame_dir, f"frame_{i}.jpg"))

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(gray_frame, 100, 200)

    segmented_filename = os.path.join(segmented_frames_dir, f"segmented_frame_{i}.jpg")
    cv2.imwrite(segmented_filename, edges)

print("Segmentation done on all frames.")


Segmentation done on all frames.


###Step 4: Track Segmented Objects Across Frames

In [6]:
def track_motion(prev_frame, next_frame):
    flow = cv2.calcOpticalFlowFarneback(prev_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

prev_frame = cv2.imread(os.path.join(frame_dir, 'frame_0.jpg'), 0)
for i in range(1, frame_count):
    next_frame = cv2.imread(os.path.join(frame_dir, f"frame_{i}.jpg"), 0)

    flow = track_motion(prev_frame, next_frame)

    prev_frame = next_frame


###Step 5: Scene Cut Detection

In [7]:
scene_cut_frames = []
threshold = 10

for i in range(1, frame_count):
    prev_frame = cv2.imread(os.path.join(frame_dir, f"frame_{i-1}.jpg"))
    next_frame = cv2.imread(os.path.join(frame_dir, f"frame_{i}.jpg"))

    diff = cv2.absdiff(prev_frame, next_frame)
    mean_diff = np.mean(diff)

    print(f"Mean diff for frame {i-1} and frame {i}: {mean_diff}")

    if mean_diff > threshold:
        scene_cut_frames.append(i)

print(f"Scene cuts detected at frames: {scene_cut_frames}")


Mean diff for frame 0 and frame 1: 3.476213509516461
Mean diff for frame 1 and frame 2: 3.2351302083333335
Mean diff for frame 2 and frame 3: 3.3925417952674897
Mean diff for frame 3 and frame 4: 3.1961295974794237
Mean diff for frame 4 and frame 5: 3.415804558899177
Mean diff for frame 5 and frame 6: 3.735236143261317
Mean diff for frame 6 and frame 7: 3.7291063850308643
Mean diff for frame 7 and frame 8: 3.7750226658950616
Mean diff for frame 8 and frame 9: 4.680049350565843
Mean diff for frame 9 and frame 10: 2.883591177983539
Mean diff for frame 10 and frame 11: 3.725033275462963
Mean diff for frame 11 and frame 12: 4.019447659465021
Mean diff for frame 12 and frame 13: 4.593113425925926
Mean diff for frame 13 and frame 14: 3.7913326260288067
Mean diff for frame 14 and frame 15: 3.3045169431584362
Mean diff for frame 15 and frame 16: 3.6466541280864195
Mean diff for frame 16 and frame 17: 4.422262249228395
Mean diff for frame 17 and frame 18: 3.38349135159465
Mean diff for frame 18

###Step 6: Mark Scene Cuts

In [8]:
for frame_idx in scene_cut_frames:
    frame = cv2.imread(os.path.join(frame_dir, f"frame_{frame_idx}.jpg"))

    marked_frame = cv2.rectangle(frame, (10, 10), (frame.shape[1]-10, frame.shape[0]-10), (0, 0, 255), 5)

    marked_filename = os.path.join(segmented_frames_dir, f"marked_frame_{frame_idx}.jpg")
    cv2.imwrite(marked_filename, marked_frame)

print("Scene cuts marked and saved.")


Scene cuts marked and saved.


###Step 7: Result Visualization

In [9]:
for i in scene_cut_frames[:5]:
    frame = cv2.imread(os.path.join(segmented_frames_dir, f"segmented_frame_{i}.jpg"))

    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title(f"Scene Cut Frame: {i}")
    plt.show()
